In [1]:
import torch
import torch.nn as nn
import numpy as np
import argparse
import torch_gnn.utils
import torch_gnn.dataloader
from torch_gnn.gnn_wrapper import GNNWrapper, SemiSupGNNWrapper
import matplotlib.pyplot as plt
import networkx as nx
import json
from tqdm import tqdm
import pandas as pd

In [2]:
df_test = pd.read_csv('../data/test.csv')
print(df_test.shape)

(43560, 3)


In [4]:
def convert_string_to_tensor(string):
    return string.replace('[', '').replace(']', '').replace(' ', '').split(',')

def get_x(vectors):
    x = np.zeros((len(vectors), 201))
    for i,vector in tqdm(enumerate(vectors)):
        #print(int(vector))
        x[i,0] = int(vector)
        x[i,1:] = np.array(convert_string_to_tensor(vectors[vector]['vector']))
        #x.append(convert_string_to_tensor(vectors[vector]['vector']))
        #x[-1].append(int(vector))
        #print(x)
        #break
    return np.array(x, dtype=np.float32)

In [6]:
with open("../data/Vectors_and_clusters.json") as file:
    vectors = json.load(file)

x = get_x(vectors)
print(x.shape)
print(x[:10])

624181it [01:06, 9391.27it/s] 


(624181, 201)
[[ 3.6030000e+03  2.0776278e-01  3.1964138e-01 ...  9.5893174e-02
   7.7129021e-02 -9.0744056e-02]
 [ 7.3830000e+03  2.6104581e-01  1.2609364e-01 ...  1.9141915e-01
  -4.4091266e-02 -3.9251558e-03]
 [ 1.4309000e+04  2.3514135e-01  2.4830784e-01 ...  4.2272650e-02
   1.6477853e-01 -8.3062634e-02]
 ...
 [ 4.9226000e+04  1.6412792e-01  2.6631457e-01 ...  1.0477072e-02
   4.1543197e-02 -5.5518728e-02]
 [ 4.9575000e+04  3.2011482e-01  2.2751243e-01 ...  1.1337683e-01
  -5.6078650e-02 -4.3000844e-03]
 [ 5.4153000e+04  6.0976766e-02  2.9971105e-01 ...  8.6528830e-02
   1.7097486e-02  5.7324026e-02]]


In [20]:
num = x[:,0].astype(int)
vec = x[:,1:]
print(num)
print(vec)

[      3603       7383      14309 ... 2908480256 2908495104 2908510976]
[[ 0.20776278  0.31964138  0.13893002 ...  0.09589317  0.07712902
  -0.09074406]
 [ 0.2610458   0.12609364  0.10306707 ...  0.19141915 -0.04409127
  -0.00392516]
 [ 0.23514135  0.24830784  0.26340294 ...  0.04227265  0.16477853
  -0.08306263]
 ...
 [ 0.24471392  0.1022459   0.03518641 ...  0.00597861 -0.05725859
  -0.02161019]
 [ 0.16219223  0.1659188   0.14239211 ... -0.02363729 -0.08755589
  -0.02170765]
 [ 0.20999359  0.08475576  0.10483249 ... -0.0883003  -0.11674631
  -0.03362756]]


In [13]:
# load the graph
G = nx.read_edgelist('../data/coauthorship.edgelist', delimiter=' ', nodetype=int)

In [14]:
edges = np.array(list(G.edges))
E = np.zeros((edges.shape[0], 3), dtype=int)
E[:,:-1] = edges
print(E[:10])

[[2002218453 1999212242          0]
 [2002218453 2032640503          0]
 [2002218453 2475931411          0]
 [2002218453 2477743428          0]
 [2002218453 2504846374          0]
 [2002218453 2597456557          0]
 [2002218453 2598017501          0]
 [2002218453 2134271654          0]
 [2002218453 2138551865          0]
 [2002218453 2145694960          0]]


In [16]:
nodes = np.array(list(G.nodes))
n = len(nodes)
print(nodes[:10])
"""for i in tqdm(range(n)):
    E = np.where(E==nodes[i], i, E)
print(E[:10])
"""

[2002218453 1999212242 2032640503 2475931411 2477743428 2504846374
 2597456557 2598017501 2134271654 2138551865]


'for i in tqdm(range(n)):\n    E = np.where(E==nodes[i], i, E)\nprint(E[:10])\n'

In [9]:
np.save('../data/E.npy', E)

In [17]:
F = np.load('../data/E.npy')
print(F.shape)

(1718164, 3)


In [21]:
print(num.shape)
print(nodes.shape)
print(vec.shape)

(624181,)
(217801,)
(624181, 200)


In [22]:
print(num[:10]) # papers
print(nodes[:10]) # authors

[ 3603  7383 14309 15548 15883 43287 43295 49226 49575 54153]
[2002218453 1999212242 2032640503 2475931411 2477743428 2504846374
 2597456557 2598017501 2134271654 2138551865]


In [30]:
#N = np.zeros((nodes.shape[0], vec.shape[1]+1))
#for i in range(n):
#    N[i,:-1] = vec[num==nodes[i]]

In [73]:
# Add features to nodes, for now with only the first paper
from numpy import genfromtxt

data = genfromtxt('../data/author_papers.txt', delimiter=':', dtype=str)
"""with open('../data/author_papers.txt', 'rb') as f:
    clean_lines = (line.replace(b':',b'-') for line in f)
    data = np.genfromtxt(clean_lines, dtype=int, delimiter='-')"""
print(data[:10])
N = np.zeros((nodes.shape[0], vec.shape[1]+1))
#print(np.min(num))
for i in tqdm(range(n)):
    author = int(data[i,0])
    papers = np.array(data[i,1].split('-'), dtype=int)
    print(papers)
    if np.sum(np.array([num==paper for paper in papers])) == 0:
        continue
    print(i)
    p = -1
    for j in range(len(papers)):
        if np.sum(num==papers[j]) == 1:
            p = j
            break
    N[nodes==author,:-1] = vec[np.argmax(num==papers[p])] # choice: 1st article
    #print(i, N[nodes==author,:])

np.save('../data/N.npy', N)

[['1036332' '1510273386-1827736641-1588673897-2252711322-2123653597']
 ['1101850' '133459021-179719743-2111787673-2126488676-31838995']
 ['1336878' '2122092249-2132109814-2100271871-2065672539-2036413831']
 ['1515524' '2141827797-2127085795-2013547785-2138529788-1994863898']
 ['1606427' '1907724546']
 ['2728936' '2114261446-2042751882-1912205781-2059913822-1905787154']
 ['3720290' '2034815413-1943970375']
 ['4265922' '2158924658-1974358662-2171008951-2143116731-2098197194']
 ['4631324' '971425015-1005190365-2775011729-1498234799-2724553065']
 ['4652436' '2029513180-1578614537-2081547332-2464290036-2164356097']]


  0%|          | 14/217801 [00:00<26:03, 139.33it/s]

[1510273386 1827736641 1588673897 2252711322 2123653597]
[ 133459021  179719743 2111787673 2126488676   31838995]
[2122092249 2132109814 2100271871 2065672539 2036413831]
[2141827797 2127085795 2013547785 2138529788 1994863898]
[1907724546]
[2114261446 2042751882 1912205781 2059913822 1905787154]
[2034815413 1943970375]
[2158924658 1974358662 2171008951 2143116731 2098197194]
[ 971425015 1005190365 2775011729 1498234799 2724553065]
[2029513180 1578614537 2081547332 2464290036 2164356097]
[2105079611 1992208760 2279367517 2426248896 2621319086]
[2114019248 2093052046 2028390205 1964770860 1992919395]
[2107960133 2396074414 2129324673 2129413054 2102857011]
[1816889752 2121942750 2022175747 2160727671 2056356307]
[1502855729 1986976921 2033310317 1482478137 2070163745]
[2113072427 2118676866 1571134607 1533950051 2108893718]
[2250322698 1998727264 2534937603 2362118473 1973707044]
[2012580531 2101108259 2060346657 2502258425 2152204876]
[1991858502 1998285060 2017661493 2022175859 211891

  0%|          | 42/217801 [00:00<27:44, 130.81it/s]

[2043509228 2074682587 1516646538 2073324778 1991213319]
[2099117796 1775249800 2163356528 2163356528 2136926356]
[2042903050 2345834824 1523757897 2529231049 1537393433]
[2061496053 2009357736 2017052870 2101668373 1979357511]
[1916347105 2200417596 1602822231 1845319420 2278091522]
[2145929661 2159740793 1970935025 2127899917 2146434055]
[1861904931 1532705157 2002713980 2034120147 2165763959]
[2114332313  413048612 2078181413 2018191967   56964303]
[2574017551 2514974017 2091939272 1519699895 2005886648]
[2166743687 1996836135 2021228972 1543770286 2102874665]
[2034794977 2027310126 2141925722 2443352794 2067384246]
[1981663600 1553781048 1541262581 1559076720   39416990]
[2519880423 2120474977  247037805 2621222911  311257967]
[2168884369 1482261514 2132934597 1791702618 2146288980]
[2151417693 2121325957 2129780509 1601355778 1563722977]
[2127113848 2110503030 2156467326 2130602336 2111748568]
[1673461548]
[2120119918 2111760412 1985654543 2110197150 1513257410]
[2137223451 196442

...

  1%|          | 1407/217801 [00:11<29:00, 124.34it/s]

[1579049907 2244083715 1522994958 1594536462 1580769899]
[2098449997 2032507921 2109813080 2094788431 2142784426]
[2146095405 2124320287 2086144884 2044306109 2055510968]
[1645021839 2052124310 1574516252 2022749141  122954271]
[1574681435 1033787497 2409357721 1568294368 1585418846]
[ 181507321 2510462315  172839780 1480191585 2161547988]
[2489225573  159547535 2064005706 2508631855 2169593029]
[2066633870 2337558443 1992316446   59467864 1987011436]
1392
[2035850477 2044977944 2121598047 1445605299  162238120]
[2130724591 1573210599 2167084044 2304832331 2136029563]
[2153465044 2146346972 2108780087 2007824828  198367286]
[2105514582 1514361308 2068101455 1515357878 2792527638]
[2141931308 1931463589  226134553 1778865519 2140687800]
[1804888644 2124439852 2161969976 2131958595 2611875969]
[1550614927 2161558926 2054120547 2149417407 1517798700]
[2185497811 1967073859 1607018412 2016706173 2559987397]
[1505885302  114967581 2007247913 1894339932  123062047]
[1777573857 1812716701 180

  1%|          | 1421/217801 [00:11<28:14, 127.70it/s]

[1974719543 2118817344 2098975839 2148958130 2129088316]
[2164674456 2137070670 2106142943 1998871470 2142444070]
[2132967511 2119209999 1528515814]
[2012869797 1524911570 2022805940 2075937957 2162437555]
[2479975364 2485074437 2115409852 2061321794 2123359309]
[1991056282 2146237557 2129687578 2165339408 1969137703]
[2030700013 1965573334 2023733790 2081251812 2094112598]
[1990832096 2103817093 2103877122 1997147891 2135682468]
[1984272814 1884998714 1497215244   67557030  158455561]
[2000576071 2247697177 1535143292 1515658647 1836481518]


KeyboardInterrupt: 

In [82]:
np.sum(num==2029513180)

0

In [79]:
print(num.dtype)

int64


In [105]:
M = np.load('../data/N.npy')
print(M.shape)

(217801, 201)


In [106]:
M = M!=0
M = M.sum(axis=1)
M = M>0
print(M.shape)

(217801,)


In [94]:
print(np.sum(M))
print(len(M))
print(np.sum(M)/len(M))

12649
217801
0.05807595006450843


In [68]:
print(len(num))
print(N.shape[0])

624181
217801


In [63]:
import pandas as pd

In [33]:
# dataset creation
df_train = pd.read_csv('../data/train.csv', dtype={'author': np.int64, 'hindex': np.float32})
n_train = df_train.shape[0]
labels = np.zeros(n_train)
for i in tqdm(range(n_train)):
    if np.sum(df_train['author']==nodes[i]) == 0:
        continue
    labels[i] = df_train[df_train['author']==nodes[i]]['hindex']
print(labels)
print(np.sum(labels>0))
np.save('../data/labels.npy', labels)

  4%|▍         | 6589/174241 [00:04<02:03, 1361.61it/s]


KeyboardInterrupt: 

In [34]:
labels = np.load('../data/labels.npy')
print(len(labels))

174241


In [35]:
# read test data
df_test = pd.read_csv('../data/test.csv', dtype={'author': np.int64})
n_test = df_test.shape[0]
print(df_test)

       Unnamed: 0      author  hindex
0               0   915630815     NaN
1               1  1236455448     NaN
2               2  2694593333     NaN
3               3  2137926699     NaN
4               4  2883694285     NaN
...           ...         ...     ...
43555       43555  2145559725     NaN
43556       43556  2168342616     NaN
43557       43557  2162797290     NaN
43558       43558   294576894     NaN
43559       43559  2508925065     NaN

[43560 rows x 3 columns]


In [112]:
cfg = GNNWrapper.Config()
cfg.use_cuda = True
cfg.device = torch_gnn.utils.prepare_device(n_gpu_use=1, gpu_id=0)
cfg.tensorboard = False
cfg.epochs = 500

cfg.activation = nn.Tanh()
cfg.state_transition_hidden_dims = [5,]
cfg.output_function_hidden_dims = [5]
cfg.state_dim = 5
cfg.max_iterations = 50
cfg.convergence_threshold = 0.01
cfg.graph_based = False
cfg.log_interval = 10
cfg.task_type = "semisupervised"
cfg.lrw = 0.001

Executing on device:  cpu


In [113]:
Ed = []
for i in tqdm(range(E.shape[0])):
    if E[i,0] < 50 and E[i,1] < 50:
        Ed.append(E[i,:])
Ed = np.array(Ed)
#print(Ed)

100%|██████████| 1718164/1718164 [00:00<00:00, 1864428.82it/s]


In [114]:
print(Ed.shape)

(211, 3)


In [115]:
# model creation
model = GNNWrapper(cfg)

dset = torch_gnn.dataloader.from_EN_to_GNN(Ed, N[:50], targets=labels[:50], aggregation_type="sum", sparse_matrix=True)  # generate the dataset

model(dset)  # dataset initalization into the GNN

In [111]:
# training code
for epoch in tqdm(range(1, cfg.epochs + 1)):
    model.train_step(epoch)

    if epoch % 10 == 0:
        model.test_step(epoch)

  0%|          | 0/500 [00:00<?, ?it/s]


RuntimeError: shape '[1, 1, 67]' is invalid for input of size 50